In [7]:
from casadi import SX, inv, vertcat, horzcat, Function, integrator, sin, cos, diag, skew, substitute, DM

In [20]:
# Equations of motion expressed in BODY

# 6 DOF states vectors
u = SX.sym('u')
v = SX.sym('v')
w = SX.sym('w')
v_nb = vertcat(u, v, w)  # body-fixed linear velocity

p = SX.sym('p')
q = SX.sym('q')
r = SX.sym('r')
w_nb = vertcat(p, q, r)  # body-fixed angular velocity

x_nb = vertcat(v_nb, w_nb)

m = SX.sym('m')  # Mass

I_x = SX.sym('I_x')  # moment of inertia x entry
I_y = SX.sym('I_y')  # moment of inertia y entry
I_z = SX.sym('I_z')  # moment of inertia z entry
I_zx = SX.sym('I_zx')  # product of inertia zx entry
I_yx = SX.sym('I_yx')  # product of inertia yx entry
I_zy = SX.sym('I_zy')  # product of inertia zy entry

x_g = SX.sym('x_g')  # Center of gravity, x-axis
y_g = SX.sym('y_g')  # Center of gravity, y-axis
z_g = SX.sym('z_g')  # Center of gravity, z-axis

x_b = SX.sym('x_b')  # Center of buoyancy, x-axis
y_b = SX.sym('y_b')  # Center of buoyancy, y-axis
z_b = SX.sym('z_b')  # Center of buoyancy, z-axis

X_du = SX.sym('X_du') # Added mass in surge
X_dv = SX.sym('X_dv') # coupled Added mass in surge & sway
X_dw = SX.sym('X_dw') # coupled Added mass in surge & heave
X_dp = SX.sym('X_dp') # coupled Added mass in surge & roll
X_dq = SX.sym('X_dq') # coupled Added mass in surge & pitch
X_dr = SX.sym('X_dr') # coupled Added mass in surge & pitch

Y_du = SX.sym('Y_du') # Added mass in sway & surge
Y_dv = SX.sym('Y_dv') # Added mass in sway
Y_dw = SX.sym('Y_dw') # coupled Added mass in sway & heave
Y_dp = SX.sym('Y_dp') # coupled Added mass in sway & roll
Y_dq = SX.sym('Y_dq') # coupled Added mass in sway & pitch
Y_dr = SX.sym('Y_dr') # coupled Added mass in sway & yaw

Z_du = SX.sym('Z_du') # coupled Added mass in heave & surge
Z_dv = SX.sym('Z_dv') # coupled Added mass in heave & sway
Z_dw = SX.sym('Z_dw') # Added mass in heave
Z_dp = SX.sym('Z_dp') # coupled Added mass in heave & roll
Z_dq = SX.sym('Z_dq') # coupled Added mass in heave & pitch
Z_dr = SX.sym('Z_dr') # coupled Added mass in heave & yaw

K_du = SX.sym('K_du') # coupled Added mass in roll & surge
K_dv = SX.sym('K_dv') # coupled Added mass in roll & sway
K_dw = SX.sym('K_dw') # coupled Added mass in roll & heave
K_dp = SX.sym('K_dp') # Added mass in roll
K_dq = SX.sym('K_dq') # coupled Added mass in roll & pitch
K_dr = SX.sym('K_dr') # coupled Added mass in roll & yaw

M_du = SX.sym('M_du') # coupled Added mass in pitch & surge
M_dv = SX.sym('M_dv') # coupled Added mass in pitch & sway
M_dw = SX.sym('M_dw') # coupled Added mass in pitch & heave
M_dp = SX.sym('M_dp') # coupled Added mass in pitch & roll
M_dq = SX.sym('M_dq') # Added mass in pitch
M_dr = SX.sym('M_dr') # coupled Added mass in pitch & yaw

N_du = SX.sym('N_du') # coupled Added mass in yaw & surge
N_dv = SX.sym('N_dv') # coupled Added mass in yaw & sway
N_dw = SX.sym('N_dw') # coupled Added mass in yaw & heave
N_dp = SX.sym('N_dp') # coupled Added mass in yaw & roll
N_dq = SX.sym('N_dq') # coupled Added mass in yaw & pitch
N_dr = SX.sym('N_dr') # Added mass in yaw

# Added Mass Forces in a Rotating Coordinate System
# Hydrodynamic System Inertia Matrix
_M_A = SX.sym('M_A', 6, 6)
_M_A[0, :] = horzcat(X_du, X_dv, X_dw, X_dp, X_dq, X_dr)
_M_A[1, :] = horzcat(Y_du, Y_dv, Y_dw, Y_dp, Y_dq, Y_dr)
_M_A[2, :] = horzcat(Z_du, Z_dv, Z_dw, Z_dp, Z_dq, Z_dr)
_M_A[3, :] = horzcat(K_du, K_dv, K_dw, K_dp, K_dq, K_dr)
_M_A[4, :] = horzcat(M_du, M_dv, M_dw, M_dp, M_dq, M_dr)
_M_A[5, :] = horzcat(N_du, N_dv, N_dw, N_dp, N_dq, N_dr)

M_A = -_M_A

# Hydrodynamic Coriolis and centripetal matrix
C_A = SX.zeros(6, 6)
a = M_A@x_nb
a_n = SX.sym('a_n', 3)
a_n[0] = a[0]
a_n[1] = a[1]
a_n[2] = a[2]

b_n = SX.sym('b_n', 3)
b_n[0] = a[3]
b_n[1] = a[4]
b_n[2] = a[5]

C_A[3:, :3] = -skew(a_n)
C_A[:3, 3:] = -skew(a_n)
C_A[3:, 3:] = -skew(b_n)

In [16]:
# Rigid body kinetics

# body inertia tensor
I_rb = SX.sym('I_rb', 3, 3)
I_rb[0, 0] = I_x
I_rb[1, 1] = I_y
I_rb[2, 2] = I_z
I_rb[1, 0] = -I_yx
I_rb[2, 0] = -I_zx
I_rb[2, 1] = -I_zy
I_rb[0, 1] = I_rb[1, 0]
I_rb[0, 2] = I_rb[2, 0]
I_rb[1, 2] = I_rb[2, 1]

# Rigid-Body System Inertia Matrix
M_rb = SX(6, 6)  # Create a 6x6 symbolic matrix

# Manually populate the matrix
M_rb[:3, :3] = m*SX.eye(3)
M_rb[3:, 3:] = I_rb

S_v = SX.sym('S_v', 3)  # inertia skew symmetric component
S_v[0] = m*x_g
S_v[1] = m*y_g
S_v[2] = m*z_g

S = skew(S_v)

M_rb[3:, :3] = S
M_rb[:3, 3:] = -S


# Coriolis-Centripetal Matrix from System Inertia Matrix
# Lagrangian Parameterization
C_rb = SX(6, 6)  # Create a 6x6 symbolic matrix
# Manually populate the matrix
c_m = SX(3, 3)

c_m[0, 0] = -m*(y_g*q + z_g*r)
c_m[1, 0] = m*(x_g*q - w)
c_m[2, 0] = m*(x_g*r + v)

c_m[0, 1] = m*(y_g*p + w)
c_m[1, 1] = -m*(z_g*r + x_g*p)
c_m[2, 1] = m*(y_g*r - u)

c_m[0, 2] = m*(z_g*p - v)
c_m[1, 2] = m*(z_g*q + u)
c_m[2, 2] = -m*(x_g*p + y_g*q)

c_I_v = SX.sym('c_I_v', 3)
c_I_v[0] = I_zx*r + I_yx*q - I_x*p
c_I_v[1] = I_zy*r + I_yx*p - I_y*q
c_I_v[2] = I_zy*q + I_zx*p - I_z*r
c_I = skew(c_I_v)

C_rb[:3, :3] = SX.zeros(3, 3)
C_rb[:3, 3:] = -c_m.T
C_rb[3:, :3] = c_m
C_rb[3:, 3:] = c_I

C = C_rb + C_A
C = substitute(C, y_g, 0)
C = substitute(C, I_yx, 0)
C = substitute(C, I_zy, 0)

C

SX(@1=0, @2=(m*(z_g*r)), @3=(m*((x_g*q)-w)), @4=((((((Z_du*u)+(Z_dv*v))+(Z_dw*w))+(Z_dp*p))+(Z_dq*q))+(Z_dr*r)), @5=(m*((x_g*r)+v)), @6=((((((Y_du*u)+(Y_dv*v))+(Y_dw*w))+(Y_dp*p))+(Y_dq*q))+(Y_dr*r)), @7=(m*w), @8=(m*((z_g*r)+(x_g*p))), @9=((((((X_du*u)+(X_dv*v))+(X_dw*w))+(X_dp*p))+(X_dq*q))+(X_dr*r)), @10=(m*u), @11=(m*((z_g*p)-v)), @12=(m*((z_g*q)+u)), @13=(m*(x_g*p)), @14=((I_zx*p)-(I_z*r)), @15=((((((N_du*u)+(N_dv*v))+(N_dw*w))+(N_dp*p))+(N_dq*q))+(N_dr*r)), @16=((((((M_du*u)+(M_dv*v))+(M_dw*w))+(M_dp*p))+(M_dq*q))+(M_dr*r)), @17=(I_y*q), @18=((I_zx*r)-(I_x*p)), @19=((((((K_du*u)+(K_dv*v))+(K_dw*w))+(K_dp*p))+(K_dq*q))+(K_dr*r)), 
[[@1, @1, @1, @2, (-(@3+@4)), (@6-@5)], 
 [@1, @1, @1, (@4-@7), @8, (-(@9-@10))], 
 [@1, @1, @1, (-(@11+@6)), (@9-@12), @13], 
 [(-@2), (@7-@4), (@11+@6), @1, (-(@14+@15)), (@16-@17)], 
 [(@3+@4), (-@8), (@12-@9), (@14+@15), @1, (-(@18+@19))], 
 [(@5-@6), (@9-@10), (-@13), (-(@16-@17)), (@18+@19), @1]])

In [19]:
# assuming Starboard–port symmetrical underwater vehicles yg=0 and Ixy = Iyz = 0

# mask = DM([[1, 0, 1, 0, 1, 0],
#            [0, 1, 0, 1, 0, 1],
#            [1, 0, 1, 0, 1, 0],
#            [0, 1, 0, 1, 0, 1],
#            [1, 0, 1, 0, 1, 0],
#            [0, 1, 0, 1, 0, 1]
#            ])
# Inertia Matrix ROV & AUV
_M = M_rb + M_A

# _M = substitute(_M, y_g, 0)
# _M = substitute(_M, I_yx, 0)
# _M = substitute(_M, I_zy, 0)
# M = _M*mask

# Function('M',[m, M_A[0]] ,[M])

_M


SX(@1=(m*z_g), @2=(m*y_g), @3=(m*x_g), 
[[(m-X_du), (-X_dv), (-X_dw), (-X_dp), (@1-X_dq), (-(@2+X_dr))], 
 [(-Y_du), (m-Y_dv), (-Y_dw), (-(@1+Y_dp)), (-Y_dq), (@3-Y_dr)], 
 [(-Z_du), (-Z_dv), (m-Z_dw), (@2-Z_dp), (-(@3+Z_dq)), (-Z_dr)], 
 [(-K_du), (-(@1+K_dv)), (@2-K_dw), (I_x-K_dp), (-(I_yx+K_dq)), (-(I_zx+K_dr))], 
 [(@1-M_du), (-M_dv), (-(@3+M_dw)), (-(I_yx+M_dp)), (I_y-M_dq), (-(I_zy+M_dr))], 
 [(-(@2+N_du)), (@3-N_dv), (-N_dw), (-(I_zx+N_dp)), (-(I_zy+N_dq)), (I_z-N_dr)]])

In [18]:
m

SX(m)

In [17]:
M.

TypeError: casadi._casadi.SX_get_free() takes exactly one argument (0 given)

In [25]:
# Hydrostatics of Submerged Vehicles
# restoring forces

W = SX.sym('W')  # weight
B = SX.sym('B')  # buoyancy

thet = SX.sym('thet')
phi = SX.sym('phi')

g = SX(6, 1)

g[0, 0] = (W - B)*sin(thet)
g[1, 0] = -(W - B)*cos(thet)*sin(phi)
g[2, 0] = -(W - B)*cos(thet)*cos(phi)
g[3, 0] = -(y_g*W - y_b*B)*cos(thet)*cos(phi) + \
    (z_g*W - z_b*B)*cos(thet)*sin(phi)
g[4, 0] = (z_g*W - z_b*B)*sin(thet) + (x_g*W - x_b*B)*cos(thet)*cos(phi)
g[5, 0] = -(x_g*W - x_b*B)*cos(thet)*sin(phi) - (y_g*W - y_b*B)*sin(thet)
# For neutrally buoyant vehicles W = B

In [9]:
# Damping will be nonlinear and coupled for an underwater vehicle moving in 6 DOFs at high speed

# <xUabsU>	Quadratic drag in surge

# <yVabsV>	Quadratic drag in sway

# <zWabsW>	Quadratic drag in heave

# <kPabsP>	Quadratic drag in roll

# <mQabsQ>	Quadratic drag in pitch

# <nRabsR>	Quadratic drag in yaw


# Linear Damping
# The linear damping matrix is the sum of potential damping and viscous damping matrices.
D = DM([[Xu, 0, 0, 0, 0, 0],
             [0, Yv, 0, Yp, 0, Yr],
             [0, 0, Zw, 0, Zq, 0],
             [0, Kv, 0, Kp, 0, Kr],
             [0, 0, Mw, 0, Mq, 0],
             [0, Nv, 0, Np, 0, Nr]])


SX(@1=0, 
[[(-D_0), @1, @1, @1, @1, @1], 
 [@1, (-D_7), @1, (-D_19), @1, (-D_31)], 
 [@1, @1, (-D_14), @1, (-D_26), @1], 
 [@1, (-D_9), @1, (-D_21), @1, (-D_33)], 
 [@1, @1, (-D_16), @1, (-D_28), @1], 
 [@1, (-D_11), @1, (-D_23), @1, (-D_35)]])

In [ ]:
# R_o = SX.sym('R_o',3,3)
# R_o[0,0] = cos(phi)*cos(thet)
# R_o[1,0] = sin(phi)*cos(thet)
# R_o[2,0] = -sin(thet)

# R_o[0,1] = -sin(phi)*cos(thet)